In [1]:
!pip install opendatasets
!pip install apyori
!pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5973 sha256=24eb473d79b1b9835404ad7a7452bdc24b768fda539651ab4ad70b15c3565b64
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
%pip install mlxtend --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.4 MB/s 
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [4]:
!gdown 12ixZGdrcwr_c4nVnjVEonFJLiFrbBzsD

Downloading...
From: https://drive.google.com/uc?id=12ixZGdrcwr_c4nVnjVEonFJLiFrbBzsD
To: /content/kaggle.json
100% 65.0/65.0 [00:00<00:00, 81.3kB/s]


In [5]:
# Imports

import opendatasets as od
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import association_rules, apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth

Apriori Algorithm

In [6]:
od.download('https://www.kaggle.com/datasets/mittalvasu95/the-bread-basket')

100%|██████████| 98.9k/98.9k [00:00<00:00, 10.9MB/s]

In [7]:
df = pd.read_csv('/content/the-bread-basket/bread basket.csv')

In [8]:
df.head()

,Transaction,Item,date_time,period_day,weekday_weekend
0,1,Bread,30-10-2016 09:58,morning,weekend
1,2,Scandinavian,30-10-2016 10:05,morning,weekend
2,2,Scandinavian,30-10-2016 10:05,morning,weekend
3,3,Hot chocolate,30-10-2016 10:07,morning,weekend
4,3,Jam,30-10-2016 10:07,morning,weekend


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20507 entries, 0 to 20506
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Transaction      20507 non-null  int64 
 1   Item             20507 non-null  object
 2   date_time        20507 non-null  object
 3   period_day       20507 non-null  object
 4   weekday_weekend  20507 non-null  object
dtypes: int64(1), object(4)
memory usage: 801.2+ KB


In [10]:
df.shape

(20507, 5)

In [11]:
# Number of unique items in transactions, Number of unique transactions

df.Item.nunique(), df.Transaction.nunique()

(94, 9465)

In [12]:
# Create dataframe that gives the count of each item purchased in every unique transaction

df_transactions = df.groupby(['Transaction', 'Item'])['Item'].count().reset_index(name='Frequency')
df_transactions

,Transaction,Item,Frequency
0,1,Bread,1
1,2,Scandinavian,2
2,3,Cookies,1
3,3,Hot chocolate,1
4,3,Jam,1
...,...,...,...
18882,9682,Tacos/Fajita,1
18883,9682,Tea,1
18884,9683,Coffee,1
18885,9683,Pastry,1


In [13]:
# Make a mxn matrix where m=transaction number and n=items and each cell represents whether the item was in the transaction or not

transactions = df_transactions.pivot_table(index='Transaction', columns='Item', values='Frequency', aggfunc='sum').fillna(0)
transactions.head()

Item,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,...,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Encoding cell values/frequency as 0 or 1

def binary_encode(x):
    if x == 0:
        return 0
    if x >= 1:
        return 1

transactions = transactions.applymap(binary_encode)
transactions.head()

Item,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,...,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Training aprori
frequent_sets = apriori(transactions, min_support=0.01, use_colnames=True)

/usr/local/lib/python3.7/dist-packages/mlxtend/frequent_patterns/fpcommon.py:115: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  DeprecationWarning,


In [16]:
# Rules from frequent itemset 
rules = association_rules(frequent_sets, metric='lift', min_threshold=1)

# Sort values by confidence
rules.sort_values('confidence', ascending=False, inplace=True)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
30,(Toast),(Coffee),0.033597,0.478394,0.023666,0.704403,1.472431,0.007593,1.764582
29,(Spanish Brunch),(Coffee),0.018172,0.478394,0.010882,0.598837,1.251766,0.002189,1.300235
19,(Medialuna),(Coffee),0.061807,0.478394,0.035182,0.569231,1.189878,0.005614,1.210871
23,(Pastry),(Coffee),0.086107,0.478394,0.047544,0.552147,1.154168,0.006351,1.164682
1,(Alfajores),(Coffee),0.036344,0.478394,0.019651,0.540698,1.130235,0.002264,1.135648
16,(Juice),(Coffee),0.038563,0.478394,0.020602,0.534247,1.116750,0.002154,1.119919
24,(Sandwich),(Coffee),0.071844,0.478394,0.038246,0.532353,1.112792,0.003877,1.115384
6,(Cake),(Coffee),0.103856,0.478394,0.054728,0.526958,1.101515,0.005044,1.102664
27,(Scone),(Coffee),0.034548,0.478394,0.018067,0.522936,1.093107,0.001539,1.093366
12,(Cookies),(Coffee),0.054411,0.478394,0.028209,0.518447,1.083723,0.002179,1.083174


It shows that {Coffee} -> {Toast} has the highest confidence.

FP Growth

In [17]:
od.download('https://www.kaggle.com/datasets/felipeguimares/games-sales-dataset-for-frequent-patterns')

100%|██████████| 91.5k/91.5k [00:00<00:00, 22.2MB/s]

In [18]:
# Read data
df = pd.read_csv('/content/games-sales-dataset-for-frequent-patterns/games_sales_dataset.csv')

In [19]:
df.head()

,God of War,The Last of Us,Read Dead Redemption,Minecraft,Grand Theft Auto V,Left 4 Dead,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Grand Theft Auto V,The Last of Us,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,God of War,Assassin's Creed 2,Read Dead Redemption,Left 4 Dead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Left 4 Dead,Assassin's Creed 2,Super Mario World,The Last of Us,Read Dead Redemption,The Elder Scrolls V: Skyrim,NaN,NaN,NaN,NaN,NaN,NaN
3,Left 4 Dead,Minecraft,The Last of Us,Dark Souls,Read Dead Redemption,Resident Evil 4,NaN,NaN,NaN,NaN,NaN,NaN
4,Dark Souls,Assassin's Creed 2,Grand Theft Auto V,Resident Evil 4,Super Mario World,Guitar Hero 3,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df.shape

(12525, 12)

In [21]:
# Convert every row into a list and convert the list to a numpy array for passing to TransactionEncoder

rows = df.shape[0]
cols = df.shape[1]

transaction = []
for i in range(rows):
    transaction.append([str(df.values[i,j]) for j in range(cols)])

transaction = np.array(transaction)

# Replace values with True/False. True if item is present in transaction and False otherwise
encoder = TransactionEncoder()
df_transformed = encoder.fit(transaction).transform(transaction)
df = pd.DataFrame(df_transformed, columns=encoder.columns_)

df.head()

,Assassin's Creed 2,Dark Souls,God of War,Grand Theft Auto V,Guitar Hero 3,Left 4 Dead,Minecraft,Read Dead Redemption,Resident Evil 4,Super Mario World,The Elder Scrolls V: Skyrim,The Last of Us,nan
0,False,False,False,True,False,False,False,False,False,False,False,True,True
1,True,False,True,False,False,True,False,True,False,False,False,False,True
2,True,False,False,False,False,True,False,True,False,True,True,True,True
3,False,True,False,False,False,True,True,True,True,False,False,True,True
4,True,True,False,True,True,False,False,False,True,True,False,False,True


In [22]:
# Drop nan column
df.drop(['nan'], axis=1, inplace=True)

In [26]:
# Obtain frequent items occuring together

item_sets = fpgrowth(df, min_support=0.1, use_colnames=True)
item_sets.head(10)

,support,itemsets
0,0.439840,(Grand Theft Auto V)
1,0.439760,(The Last of Us)
2,0.437525,(Read Dead Redemption)
3,0.437206,(Left 4 Dead)
4,0.433613,(God of War)
5,0.429780,(Assassin's Creed 2)
6,0.437844,(The Elder Scrolls V: Skyrim)
7,0.434651,(Super Mario World)
8,0.439281,(Minecraft)
9,0.437764,(Resident Evil 4)


In [27]:
# Create association rules for frequent items
rules = association_rules(item_sets, metric='lift', min_threshold=1)

# Sort values by confidence
rules.sort_values('confidence', ascending=False, inplace=True)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
57,"(Super Mario World, God of War)",(Resident Evil 4),0.199361,0.437764,0.100998,0.506608,1.157261,0.013725,1.139531
56,"(Super Mario World, Resident Evil 4)",(God of War),0.200000,0.433613,0.100998,0.504990,1.164611,0.014275,1.144194
58,"(Resident Evil 4, God of War)",(Super Mario World),0.205669,0.434651,0.100998,0.491071,1.129807,0.011604,1.110862
53,(God of War),(Resident Evil 4),0.433613,0.437764,0.205669,0.474314,1.083492,0.015848,1.069528
52,(Resident Evil 4),(God of War),0.437764,0.433613,0.205669,0.469816,1.083492,0.015848,1.068284
...,...,...,...,...,...,...,...,...,...
80,(Grand Theft Auto V),(Assassin's Creed 2),0.439840,0.429780,0.193054,0.438918,1.021261,0.004019,1.016286
66,(Left 4 Dead),(Assassin's Creed 2),0.437206,0.429780,0.190978,0.436815,1.016368,0.003076,1.012491
61,(God of War),"(Super Mario World, Resident Evil 4)",0.433613,0.200000,0.100998,0.232922,1.164611,0.014275,1.042919
59,(Super Mario World),"(Resident Evil 4, God of War)",0.434651,0.205669,0.100998,0.232366,1.129807,0.011604,1.034779


Confidence ({(Super Mario World, God of War)} -> {Resident Evil 4}) = 0.50 means that if there are 'x' transactions containing the antecedent, then there will be x/2 that will also contain the consequent.